In [1]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

load_dotenv("hf.env")
print("Token starts with:", os.getenv("HF_TOKEN")[:10], "...")


ModuleNotFoundError: No module named 'huggingface_hub'

In [2]:
!pip install huggingface_hub python-dotenv ipywidgets

   ---------------------------------------- 0.0/564.3 kB ? eta -:--:--
   ---------------------------------------- 564.3/564.3 kB 9.7 MB/s eta 0:00:00


In [2]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

load_dotenv("hf.env")
print("Token starts with:", os.getenv("HF_TOKEN")[:10], "...")


Token starts with: hf_hLhXdHu ...


In [3]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

load_dotenv()

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"  # free/public model
client = InferenceClient(model=MODEL_ID, token=os.getenv("HF_TOKEN"))

# sanity test
resp = client.text_generation(
    prompt="Write one short friendly sentence confirming this works.",
    max_new_tokens=50
)
print(resp)

ValueError: Model mistralai/Mistral-7B-Instruct-v0.2 is not supported for task text-generation and provider featherless-ai. Supported task: conversational.

In [26]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

load_dotenv("hf.env")

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2" # free/public model
client = InferenceClient(model=MODEL_ID, token=os.getenv("HF_TOKEN"))  #token - API key 

SYSTEM_PROMPT = "You are a concise, helpful assistant."

resp = client.chat_completion(
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": "Give me a short and crisp response to let me know this works!"}
    ],
    max_tokens=120,
)
print(resp.choices[0].message["content"])


 I'm here to help. Your interaction indicates that the communication is working effectively. Keep asking if you have any other questions or need assistance with anything else.


In [27]:
def chat(user_message: str) -> str:
    out = client.chat_completion(
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_message}
        ],
        max_tokens=256,  # 256 - ensures answers are not too long
        temperature=0.7, # medium setting
        top_p=0.95,
    )
    reply = out.choices[0].message["content"]
    return reply

# testing it
print(chat("Give me some pointers to learn AI."))

 1. Start with the basics: Learn mathematics required for AI such as linear algebra, calculus, probability, and statistics. Familiarize yourself with programming concepts using languages like Python, which is widely used in AI.

2. Understand key concepts: Learn about Machine Learning (ML), Deep Learning (DL), Neural Networks, and other AI concepts. ML algorithms include regression, decision trees, random forests, SVM, and clustering. DL focuses on neural networks and deep architectures.

3. Hands-on experience: Work on projects using libraries like TensorFlow, Scikit-learn, and Keras. Implement ML and DL models on real-world datasets.

4. Online courses and tutorials: Websites like Coursera, edX, and Kaggle offer free and paid courses on AI and ML. They provide structured learning paths and projects to practice.

5. Books: Read textbooks like "Pattern Recognition and Machine Learning" by Christopher Bishop and "Deep Learning" by Ian Goodfellow.

6. Join the community: Participate in o

In [6]:
# --- Simple chat box UI (ipywidgets) with memory ---

from ipywidgets import Text, VBox, HTML, Button, HBox
from IPython.display import display

# conversation memory (system + rolling history)
history = [{"role": "system", "content": SYSTEM_PROMPT}]

# UI elements
log = HTML(value="<b>Chat started.</b><br>", placeholder="")
input_box = Text(placeholder="Type a message and press Enter…", disabled=False)
clear_btn = Button(description="Clear", tooltip="Clear chat", button_style="", layout={"width": "90px"})

def render_line(role: str, text: str):
    prefix = "<b>You:</b> " if role == "user" else "<b>Bot:</b> "
    return prefix + text.replace("\n", "<br>") + "<br>"

def send_to_model():
    # last user message is at the end of history
    out = client.chat_completion(
        messages=history,
        max_tokens=256,
        temperature=0.7,
        top_p=0.95,
    )
    reply = out.choices[0].message["content"]
    history.append({"role": "assistant", "content": reply})
    log.value += render_line("assistant", reply)

def on_submit(change):
    msg = change["new"].strip()
    if not msg:
        return
    # show user line
    log.value += render_line("user", msg)
    # add to history
    history.append({"role": "user", "content": msg})
    # clear input first (snappier UX)
    input_box.value = ""
    input_box.disabled = True
    try:
        send_to_model()
    except Exception as e:
        log.value += render_line("assistant", f"Error: {e}")
    finally:
        input_box.disabled = False

def on_clear(_):
    # reset memory but keep system prompt
    del history[:]
    history.append({"role": "system", "content": SYSTEM_PROMPT})
    log.value = "<b>Chat cleared.</b><br>"

input_box.observe(on_submit, names="value")
controls = HBox([clear_btn])
clear_btn.on_click(on_clear)

display(VBox([controls, log, input_box]))

In [7]:
import sys, subprocess, os
print("Python:", sys.version)
print("Kernel:", sys.executable)

# Show widget packages
def show(pkg):
    try:
        out = subprocess.check_output([sys.executable, "-m", "pip", "show", pkg], text=True)
        print(f"\n=== {pkg} ===\n" + out)
    except subprocess.CalledProcessError:
        print(f"\n=== {pkg} ===\n(not installed)")

for p in ["ipywidgets", "jupyterlab-widgets", "widgetsnbextension"]:
    show(p)

Python: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Kernel: E:\Anaconda3\python.exe

=== ipywidgets ===
Name: ipywidgets
Version: 7.8.1
Summary: IPython HTML widgets for Jupyter
Home-page: http://ipython.org
Author: IPython Development Team
Author-email: ipython-dev@scipy.org
License: BSD
Location: E:\Anaconda3\Lib\site-packages
Requires: comm, ipython, ipython-genutils, jupyterlab-widgets, traitlets, widgetsnbextension
Required-by: jupyter


=== jupyterlab-widgets ===
Name: jupyterlab-widgets
Version: 1.0.0
Summary: A JupyterLab extension.
Home-page: https://github.com/jupyter-widgets/ipywidgets
Author: Jupyter Development Team
Author-email: 
License: BSD-3-Clause
Location: E:\Anaconda3\Lib\site-packages
Requires: 
Required-by: ipywidgets


=== widgetsnbextension ===
Name: widgetsnbextension
Version: 3.6.6
Summary: IPython HTML widgets for Jupyter
Home-page: http://ipython.org
Author: IPython Development Team
Author-email: ipython-de

In [8]:
!pip install -U ipywidgets jupyterlab-widgets widgetsnbextension

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 882.6 kB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.2 MB 931.2 kB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.2 MB 931.2 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.2 MB 728.9 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.2 MB 728.9 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.2 MB 728.9 kB/s eta 0:00:02
   ----------------------- ---------------- 1.3/2.2 MB 604.4 kB/s eta 0:00:02
   ----------------------- ---------------- 1.3/2.2 MB 604.4 kB/s eta 0:00:02
   ---------------------------- ----------- 1.6/2.2 MB 635.3 kB/s eta 0:00:01
   -------------------

In [9]:
from ipywidgets import Text
Text(placeholder="If you see this textbox, widgets are now working ✅")

Text(value='', placeholder='If you see this textbox, widgets are now working ✅')

In [10]:
import sys, subprocess

# Install (safe to rerun), then enable the nbextension for classic Notebook
subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "install", "--py", "widgetsnbextension", "--sys-prefix"])
subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "enable",  "--py", "widgetsnbextension", "--sys-prefix"])

# Show status
subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "list"])

CalledProcessError: Command '['E:\\Anaconda3\\python.exe', '-m', 'jupyter', 'nbextension', 'install', '--py', 'widgetsnbextension', '--sys-prefix']' returned non-zero exit status 1.

# Assistant
This error occurs when the Jupyter command to install the widget extension fails. This could be due to permission issues, a corrupted Anaconda installation, or missing dependencies.

The error specifically happens when trying to run:

In [11]:
subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "install", "--py", "widgetsnbextension", "--sys-prefix"])

CalledProcessError: Command '['E:\\Anaconda3\\python.exe', '-m', 'jupyter', 'nbextension', 'install', '--py', 'widgetsnbextension', '--sys-prefix']' returned non-zero exit status 1.


Would you like me to provide a corrected approach to install the widget extension?

# User
yes

In [12]:
# Assistant
import sys, subprocess

try:
    # First, make sure the widgetsnbextension package is installed
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "widgetsnbextension"])
    
    # Then install and enable the nbextension for classic Notebook
    # Using pip instead of direct nbextension commands can help avoid permission issues
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "ipywidgets"])
    
    # If you still want to use the nbextension commands, try without --sys-prefix first
    subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "enable", "--py", "widgetsnbextension"])
    
    # Show status
    subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "list"])
    
    print("Widget extension successfully installed and enabled!")
except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e}")
    print("Try running this code in an Anaconda Prompt with administrator privileges")

Error occurred: Command '['E:\\Anaconda3\\python.exe', '-m', 'jupyter', 'nbextension', 'enable', '--py', 'widgetsnbextension']' returned non-zero exit status 1.
Try running this code in an Anaconda Prompt with administrator privileges


In [13]:
import sys, subprocess

try:
    # Install packages with --user flag to avoid permission issues
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", "--upgrade", "widgetsnbextension"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", "--upgrade", "ipywidgets"])
    
    # Enable the extension with --user flag
    subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "enable", "--py", "widgetsnbextension", "--user"])
    
    # Show status
    subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "list"])
    
    print("Widget extension successfully installed and enabled!")
except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e}")
    print("If this still fails, try installing manually from an Anaconda Prompt:")
    print("pip install --user ipywidgets")
    print("jupyter nbextension enable --py widgetsnbextension --user")

Error occurred: Command '['E:\\Anaconda3\\python.exe', '-m', 'jupyter', 'nbextension', 'enable', '--py', 'widgetsnbextension', '--user']' returned non-zero exit status 1.
If this still fails, try installing manually from an Anaconda Prompt:
pip install --user ipywidgets
jupyter nbextension enable --py widgetsnbextension --user


In [14]:
import sys, subprocess

# Try user-level install/enable (no admin rights needed)
subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "install", "--py", "widgetsnbextension", "--user"])
subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "enable",  "--py", "widgetsnbextension", "--user"])

# Show status
subprocess.call([sys.executable, "-m", "jupyter", "nbextension", "list"])

CalledProcessError: Command '['E:\\Anaconda3\\python.exe', '-m', 'jupyter', 'nbextension', 'install', '--py', 'widgetsnbextension', '--user']' returned non-zero exit status 1.